In [16]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [17]:
xx = QQ['xx'].0

In [18]:
Q.<alpha> = NumberField(xx^2+3)

In [19]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)

In [20]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [21]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [22]:
line = Line([y, z])

In [23]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [24]:
SE18 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f:c*(3*c-e)/(c+e)}).subs({e : alpha*c})

In [25]:
len(SE18.find_admissible_permutations())

1296

In [26]:
%time adm_SE18 = SE18.find_admissible_projectivities()

CPU times: user 1.1 s, sys: 130 ms, total: 1.23 s
Wall time: 10.9 s


In [27]:
len(adm_SE18)

648

In [28]:
%time simm_SE18 = SE18.find_simmetries(adm_SE18)

CPU times: user 912 ms, sys: 113 ms, total: 1.02 s
Wall time: 1.51 s


In [29]:
len(simm_SE18)

648

In [15]:
G_SE18 = Group(simm_SE18)

In [16]:
#%time G_SE18.is_group()

In [17]:
G_SE18

Non abelian group of order 648

In [18]:
G_SE18.get_order()

648

In [19]:
G_SE18.is_abelian()

False

In [20]:
G_SE18.get_divisors_of_order()

[1, 2, 3, 4, 6, 8, 9, 12, 18, 24, 27, 36, 54, 72, 81, 108, 162, 216, 324, 648]

In [21]:
perms = []
for el in G_SE18:
    perms.append(Permutation(apply_proj_to_eck(el, SE18.eckardt_points)).to_cycles())

In [22]:
gp = PermutationGroup(perms)

In [23]:
gp.order()

648

In [24]:
SE18

(6*alpha + 6)*x^2*y + (-6*alpha - 6)*x*y^2 + (-4*alpha - 4)*x^2*z + (4*alpha + 4)*x*y*z + (-2*alpha - 2)*x*z^2 + (-4*alpha - 4)*x*y*t + (4*alpha + 4)*y^2*t + (2*alpha + 2)*y*t^2

In [25]:
gp.structure_description()

'(((C3 x C3 x C3) : (C2 x C2)) : C3) : C2'

In [26]:
nor_subs = gp.normal_subgroups()

In [27]:
nor_sub = [el for el in nor_subs if el.order()==27][0]

In [28]:
gp.quotient(nor_sub).is_isomorphic(SymmetricGroup(4))

True

In [29]:
gp.quotient(nor_sub).minimal_generating_set()

[(1,9,3)(2,13,6)(4,18,19)(5,23,10)(7,21,22)(8,24,14)(11,12,17)(15,16,20),
 (1,8,4,16)(2,5,7,12)(3,21,10,24)(6,18,14,23)(9,15,17,22)(11,20,19,13)]

In [30]:
nor_sub.structure_description()

'C3 x C3 x C3'

In [31]:
SE18.eckardt_points_labels

[['E1', 'G3', 'F13'],
 ['E1', 'G4', 'F14'],
 ['E2', 'G3', 'F23'],
 ['E2', 'G4', 'F24'],
 ['E3', 'G5', 'F35'],
 ['E3', 'G6', 'F36'],
 ['E4', 'G5', 'F45'],
 ['E4', 'G6', 'F46'],
 ['E5', 'G1', 'F15'],
 ['E5', 'G2', 'F25'],
 ['E6', 'G1', 'F16'],
 ['E6', 'G2', 'F26'],
 ['F12', 'F35', 'F46'],
 ['F12', 'F36', 'F45'],
 ['F13', 'F24', 'F56'],
 ['F14', 'F23', 'F56'],
 ['F15', 'F26', 'F34'],
 ['F16', 'F25', 'F34']]

In [32]:
lines_three_eck = []
for i in range(16):
    for j in range(i+1, 17):
        for k in range(j+1, 18):
            point1 = SE18.eckardt_points[i]
            point2 = SE18.eckardt_points[j]
            point3 = SE18.eckardt_points[k]   
            line12_pl = Point(matrix([point1, point2]).minors(2))
            line23_pl = Point(matrix([point2, point3]).minors(2))
            if line12_pl == line23_pl and line12_pl not in [line.plucker for line in lines_three_eck]:
                planes = get_planes(line12_pl)
                lines_three_eck.append(Line(planes,[point1, point2], line12_pl))

In [33]:
len(lines_three_eck)

42

In [34]:
def get_all_lines_incident_to_line(line, lines):
    return [other_line for other_line in lines if line.are_incident(other_line)]

In [34]:
A = matrix([[line1.are_incident(line2) for line2 in lines_three_eck] for line1 in lines_three_eck])

In [35]:
for i in range(42-12):
    for j in range(42-12):
        if 0 not in A.submatrix(i,j, 12, 12).list():
            print(i,j)

In [36]:
sum([A.row(i) for i in range(42)])

(23, 21, 21, 21, 21, 21, 21, 23, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 23, 21, 21, 23, 21, 21, 21, 21, 21, 21, 23, 23, 21, 21, 21, 21)